## 第3章: 正規表現

time required: 3hours

---

Wikipediaの記事を以下のフォーマットで書き出したファイルjawiki-country.json.gzがある．

- 1行に1記事の情報がJSON形式で格納される
- 各行には記事名が"title"キーに，記事本文が"text"キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される
- ファイル全体はgzipで圧縮される

以下の処理を行うプログラムを作成せよ．

In [1]:
%%bash

# get file
if [ ! -f jawiki-country.json ]; then
    wget http://www.cl.ecei.tohoku.ac.jp/nlp100/data/jawiki-country.json.gz
    gunzip -k jawiki-country.json.gz
fi

## 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [2]:
import json

data = {}
with open('jawiki-country.json') as f:
    for line in f.readlines():
        j = json.loads(line.strip())
        data[j['title']] = j['text']

len(data)

206

In [3]:
print(data[u'イギリス'][:1000])

{{redirect|UK}}
{{基礎情報 国
|略名 = イギリス
|日本語国名 = グレートブリテン及び北アイルランド連合王国
|公式国名 = {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）<br/>
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）<br/>
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）<br/>
*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）<br/>
*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）<br/>
**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>
|国旗画像 = Flag of the United Kingdom.svg
|国章画像 = [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]
|国章リンク = （[[イギリスの国章|国章]]）
|標語 = {{lang|fr|Dieu et mon droit}}<br/>（[[フランス語]]:神と私の権利）
|国歌 = [[女王陛下万歳|神よ女王陛下を守り給え]]
|位置画像 = Location_UK_

## 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ．

In [4]:
for line in data[u'イギリス'].split('\n'):
    if line.startswith('[[Category:'):
        print(line)

[[Category:イギリス|*]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国]]
[[Category:海洋国家]]
[[Category:君主国]]
[[Category:島国|くれいとふりてん]]
[[Category:1801年に設立された州・地域]]


## 22. カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [5]:
import re
for line in data[u'イギリス'].split('\n'):
    m = re.match(r"^\[\[Category:([^\]|]*)(|.*)?\]\]$", line)
    if m:
        g = m.group(1, 2)
        print(g[0])

イギリス
英連邦王国
G8加盟国
欧州連合加盟国
海洋国家
君主国
島国
1801年に設立された州・地域


## 23. セクション構造
記事中に含まれるセクション名とそのレベル（例えば"== セクション名 =="なら1）を表示せよ．

In [6]:
for line in data[u'イギリス'].split('\n'):
    m = re.match(r"^(={2,4})(.*)(\1)$", line)
    if m:
        g = m.group(1, 2, 3)
        print('%s %s (level %d)' % ('  '*len(g[0])+'- ', g[1], len(g[0])))

    -  国名 (level 2)
    -  歴史 (level 2)
    -  地理 (level 2)
      -  気候 (level 3)
    -  政治 (level 2)
    -  外交と軍事 (level 2)
    -  地方行政区分 (level 2)
      -  主要都市 (level 3)
    -  科学技術 (level 2)
    -  経済 (level 2)
      -  鉱業 (level 3)
      -  農業 (level 3)
      -  貿易 (level 3)
      -  通貨 (level 3)
      -  企業 (level 3)
    -  交通 (level 2)
      -  道路 (level 3)
      -  鉄道 (level 3)
      -  海運 (level 3)
      -  航空 (level 3)
    -  通信 (level 2)
    -  国民 (level 2)
      -  言語 (level 3)
      -  宗教 (level 3)
      -   婚姻  (level 3)
      -  教育 (level 3)
    -  文化 (level 2)
      -  食文化 (level 3)
      -  文学 (level 3)
      -   哲学  (level 3)
      -  音楽 (level 3)
        -  イギリスのポピュラー音楽 (level 4)
      -  映画 (level 3)
      -  コメディ (level 3)
      -  国花 (level 3)
      -  世界遺産 (level 3)
      -  祝祭日 (level 3)
    -  スポーツ (level 2)
      -  サッカー (level 3)
      -  競馬 (level 3)
      -  モータースポーツ (level 3)
    -  脚注 (level 2)
    -  関連項目 (level 2)
    -  外部リンク (level 2)


## 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ．

In [7]:
media_ext = ['jpg', 'png', 'svg']
for line in data[u'イギリス'].split('\n'):
    m = re.match(r"^\[\[File:([^|]*)(.*)?\]\]$", line)
    if m:
        g = m.group(1, 2)
        assert g[0].rsplit('.', 1)[-1].lower() in media_ext
        print(g[0])

Battle of Waterloo 1815.PNG
The British Empire.png
Uk topo en.jpg
BenNevis2005.jpg
Elizabeth II greets NASA GSFC employees, May 8, 2007 edit.jpg
Palace of Westminster, London - Feb 2007.jpg
David Cameron and Barack Obama at the G20 Summit in Toronto.jpg
Soldiers Trooping the Colour, 16th June 2007.jpg
Scotland Parliament Holyrood.jpg
London.bankofengland.arp.jpg
City of London skyline from London City Hall - Oct 2008.jpg
Oil platform in the North SeaPros.jpg
Eurostar at St Pancras Jan 2008.jpg
Heathrow T5.jpg
Anglospeak.svg


## 25. テンプレートの抽出
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [8]:
# regex matching nested parenthesis
#        source: http://blog.ni-ken.net/2015/04/python-regex-tips/

import regex
from collections import OrderedDict

text = data[u'イギリス']
while True:
    m = regex.match(r"(?<reg>\{(?:[^{}]+|(?&reg))*\})", text)
    if m:
        infobox25 = m.group('reg')
        if infobox25.startswith(u'{{基礎情報'):
            break
        else:
            text = text[len(infobox25)+1:]
    #else:
    #    raise Error('not found!')
    #    break

def construct_infobox_dict(infobox):
    """save data in dict"""
    dic = OrderedDict()
    for line in infobox.strip('{}').split('\n|'):
        info = line.split('=', 1)
        if len(info) == 2:
            dic[info[0].strip()] = info[1].strip()
    return dic


dic = construct_infobox_dict(infobox25)
for k, v in dic.items():
    print('%10s\t%s' % (k, v))

        略名	イギリス
     日本語国名	グレートブリテン及び北アイルランド連合王国
      公式国名	{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）<br/>
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）<br/>
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）<br/>
*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）<br/>
*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）<br/>
**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>
      国旗画像	Flag of the United Kingdom.svg
      国章画像	[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]
     国章リンク	（[[イギリスの国章|国章]]）
        標語	{{lang|fr|Dieu et mon droit}}<br/>（[[フランス語]]:神と私の権利）
        国歌	[[女王陛下万歳|神よ女王陛下を守り給え]]
      位置画像	Locati

## 26. 強調マークアップの除去
25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: マークアップ早見表）．

In [9]:
infobox26 = re.sub(r"('{2,5})([^']+)\1", r"\2", infobox25)
dic = construct_infobox_dict(infobox26)
for k, v in dic.items():
    print('%10s\t%s' % (k, v))

        略名	イギリス
     日本語国名	グレートブリテン及び北アイルランド連合王国
      公式国名	{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）<br/>
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）<br/>
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）<br/>
*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）<br/>
*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）<br/>
**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>
      国旗画像	Flag of the United Kingdom.svg
      国章画像	[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]
     国章リンク	（[[イギリスの国章|国章]]）
        標語	{{lang|fr|Dieu et mon droit}}<br/>（[[フランス語]]:神と私の権利）
        国歌	[[女王陛下万歳|神よ女王陛下を守り給え]]
      位置画像	Locati

## 27. 内部リンクの除去
26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: マークアップ早見表）．

In [10]:
infobox27 = re.sub(r"\[\[(([^|\]]+\|)*([^\]]+))\]\]", r"\3", infobox26)
dic = construct_infobox_dict(infobox27)
for k, v in dic.items():
    print('%10s\t%s' % (k, v))

        略名	イギリス
     日本語国名	グレートブリテン及び北アイルランド連合王国
      公式国名	{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（スコットランド・ゲール語）<br/>
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（ウェールズ語）<br/>
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（アイルランド語）<br/>
*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（コーンウォール語）<br/>
*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（スコットランド語）<br/>
**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>
      国旗画像	Flag of the United Kingdom.svg
      国章画像	イギリスの国章
     国章リンク	（国章）
        標語	{{lang|fr|Dieu et mon droit}}<br/>（フランス語:神と私の権利）
        国歌	神よ女王陛下を守り給え
      位置画像	Location_UK_EU_Europe_001.svg
       公用語	英語（事実上）
        首都	ロンドン
      最大都市	ロンドン
     元首等肩書	女王
     元首等氏名	エリザベス2

## 28. MediaWikiマークアップの除去
27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

In [11]:
from html.entities import html5
infobox28 = re.sub(r"\{\{(([^|}]+\|)*([^}]+))\}\}", r"\3", infobox27)
infobox28 = re.sub(r"<([^>]+)[^>]*>([^<]+)</\1>", "", infobox28)
infobox28 = re.sub(r"<([^/>]+)/>", "", infobox28)
html_symbols = re.findall(r"&[a-z]+;", infobox28)
for symbol in html_symbols:
    infobox28 = infobox28.replace(symbol, html5[symbol[1:]])
dic = construct_infobox_dict(infobox28)
for k, v in dic.items():
    print('%10s\t%s' % (k, v))

      国旗画像	Flag of the United Kingdom.svg
      国章画像	イギリスの国章
     国章リンク	（国章）
        標語	Dieu et mon droit（フランス語:神と私の権利）
        国歌	神よ女王陛下を守り給え
      位置画像	Location_UK_EU_Europe_001.svg
       公用語	英語（事実上）
        首都	ロンドン
      最大都市	ロンドン
     元首等肩書	女王
     元首等氏名	エリザベス2世
     首相等肩書	首相
     首相等氏名	デーヴィッド・キャメロン
      面積順位	76
     面積大きさ	1 E11
       面積値	244,820
      水面積率	1.3%
     人口統計年	2011
      人口順位	22
     人口大きさ	1 E7
       人口値	63,181,775
     人口密度値	246
   GDP統計年元	2012
     GDP値元	1兆5478億
 GDP統計年MER	2012
  GDP順位MER	5
   GDP値MER	2兆4337億
    GDP統計年	2012
     GDP順位	6
      GDP値	2兆3162億
     GDP/人	36,727
      建国形態	建国
     確立形態1	イングランド王国／スコットランド王国（両国とも1707年連合法まで）
    確立年月日1	927年／843年
     確立形態2	グレートブリテン王国建国（1707年連合法）
    確立年月日2	1707年
     確立形態3	グレートブリテン及びアイルランド連合王国建国（1800年連合法）
    確立年月日3	1801年
     確立形態4	現在の国号「グレートブリテン及び北アイルランド連合王国」に変更
    確立年月日4	1927年
        通貨	UKポンド (£)
     通貨コード	GBP
       時間帯	±0
       夏時間	+1
ISO 3166-1	GB / GBR
     ccTLD	.uk / .gb
    国際電話番号	44
        注記	


## 29. 国旗画像のURLを取得する
テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: MediaWiki APIのimageinfoを呼び出して，ファイル参照をURLに変換すればよい）

In [12]:
from six.moves.urllib.request import urlopen
from xml.etree.ElementTree import fromstring, tostring

url = "http://ja.wikipedia.org/w/api.php?format=xml&action=query"
url += "&titles=Image:%s&prop=imageinfo&iiprop=url" % dic[u"国旗画像"].replace(' ', '_')

xml = urlopen(url).read()
tree = fromstring(xml)
imageinfo = tree.find('.//imageinfo/*[@url]')
print(imageinfo.attrib.get("url"))

https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg


![](https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg)